In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
# load the data
columns = ['time',
 'distance',
 'easting',
 'northing',
 'ellipsoid_height',
 'latitude',
 'longitude',
 'ellipsoid_height_1',
 'roll',
 'pitch',
 'heading',
 'east_velocity',
 'north_velocity',
 'up_velocity',
 'east_sd',
 'north_sd',
 'height_sd',
 'roll_sd',
 'pitch_sd',
 'heading_sd']

imu_df = pd.read_csv("../../Data/imu_data/export_Mission_2.txt", 
                     sep='[ ]+',
                     engine='python',
                     skiprows=28,
                     header=None,
                     names=columns)

In [9]:
# Import anchor pickle
anchors = pd.read_pickle("../../Data/pickles/anchor_points_164239_just_xyz.pkl")
anchors_df = pd.DataFrame(anchors,columns=['gps_time'])

In [5]:
def slice_imu(imu_df,lidar_df,imu_time_name = 'time',lidar_time_name='gps_time'):
    #Function slices imu_df for the points that fall within the GPS timeframe of the provided lidar_df.
    
    # Determine gps timeframe for lidar
    lidar_range = (lidar_df[lidar_time_name].min(),lidar_df[lidar_time_name].max())
    print("Timespan of LiDAR data: {:2.2f} seconds".format(lidar_range[1]-lidar_range[0]))
    
    # Slide of IMU data within lidar_range
    in_slice = (imu_df[imu_time_name]>=lidar_range[0]) & (imu_df[imu_time_name]<=lidar_range[1])
    imu_slice = imu_df[in_slice]
    print("IMU points in timespan: {:d}".format(imu_slice.shape[0]))
    return imu_slice

In [6]:
# Create IMU slice
imu_slice = slice_imu(imu_df,anchors_df)

Timespan of LiDAR data: 57.29 seconds
IMU points in timespan: 11459


In [12]:
imu_df.head()

,time,distance,easting,northing,ellipsoid_height,latitude,longitude,ellipsoid_height_1,roll,pitch,heading,east_velocity,north_velocity,up_velocity,east_sd,north_sd,height_sd,roll_sd,pitch_sd,heading_sd
0,576186.00101,0.000,563810.653,4496470.594,-22.876,40.616597,-74.245624,-22.876,-1.654,-7.485,255.379,-19.906,-4.809,0.650,0.085,0.069,0.105,0.02,0.019,0.047
1,576186.00601,0.102,563810.553,4496470.569,-22.873,40.616597,-74.245625,-22.873,-1.648,-7.486,255.379,-19.914,-4.810,0.663,0.085,0.069,0.105,0.02,0.019,0.047
2,576186.01101,17.758,563827.660,4496474.878,-23.427,40.616634,-74.245423,-23.427,-1.642,-7.490,255.379,-19.921,-4.810,0.673,0.085,0.069,0.105,0.02,0.019,0.047
3,576186.01601,17.861,563827.560,4496474.853,-23.424,40.616634,-74.245424,-23.424,-1.636,-7.496,255.379,-19.927,-4.810,0.676,0.085,0.069,0.105,0.02,0.019,0.047
4,576186.02101,17.963,563827.461,4496474.828,-23.421,40.616634,-74.245425,-23.421,-1.632,-7.502,255.378,-19.933,-4.809,0.679,0.085,0.069,0.105,0.02,0.019,0.047


In [ ]:
%time
a = np.zeros(len(anchors_df))
for i,time in enumerate(imu_slice['time']):
    a = [time if time <= pt else a[i] for i,pt in enumerate(anchors_df['gps_time'])]
    if i%10 == 0:
        print("Iteration count: ",i)
    
anchors_df['imu_time'] = a

In [ ]:
imu_slice[imu_slice['time']==a[1]]

In [ ]:
a[:5]

In [ ]:
anchor_df.head()

In [ ]:
# Plot some IMU data
keys = ['time','distance','latitude','longitude','ellipsoid_height','roll','pitch','heading','up_velocity']
font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 16,
        }

plt.figure(figsize=(20,25))
for i, key in enumerate(keys):
    plt.subplot(3,3, (i+1))
    plt.plot(imu_slice['time'],imu_slice[key],'xr')
    plt.title(str(key),fontdict=font)
    if i >= 6:
        plt.xlabel("GPS Time (s)",fontdict=font)

In [ ]:
field = 'pitch'
plt.figure(figsize=(20,25))
plt.plot(imu_slice['time'],imu_slice[field],'xr')
plt.ylabel(field)
plt.xlabel("GPS Time")

In [ ]:
imu_slice['time'].max()-imu_slice['time'].min()

In [ ]:
anchors_range[0]-anchors_range[1]

In [ ]:
imu_slice.shape

### Find a specific Lat/Lon
40°38'38.8"N 74°01'33.0"W

In [31]:
def convert_lat_lon_to_dec(degrees,minutes,seconds):
    return degrees + (minutes/60) + (seconds/3600)
lat = convert_lat_lon_to_dec(40,38,38.8)
lon = convert_lat_lon_to_dec(74,1,33.0)*-1
print("Latitude: {:2.4f}".format(lat))
print("Longitude: {:2.4f}".format(lon))

Latitude: 40.6441
Longitude: -74.0258


In [32]:
# Sort by nearest lat-lon ()
np.argmin(abs(imu_df['latitude']+imu_df['longitude']-lat-lon))

//anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.
  return getattr(obj, method)(*args, **kwds)


222005

In [28]:
imu_df['latitude'][222005]

40.64254814

In [33]:
imu_df['longitude'][222005]

-74.01754882

In [30]:
imu_df.head()

,time,distance,easting,northing,ellipsoid_height,latitude,longitude,ellipsoid_height_1,roll,pitch,heading,east_velocity,north_velocity,up_velocity,east_sd,north_sd,height_sd,roll_sd,pitch_sd,heading_sd
0,576186.00101,0.000,563810.653,4496470.594,-22.876,40.616597,-74.245624,-22.876,-1.654,-7.485,255.379,-19.906,-4.809,0.650,0.085,0.069,0.105,0.02,0.019,0.047
1,576186.00601,0.102,563810.553,4496470.569,-22.873,40.616597,-74.245625,-22.873,-1.648,-7.486,255.379,-19.914,-4.810,0.663,0.085,0.069,0.105,0.02,0.019,0.047
2,576186.01101,17.758,563827.660,4496474.878,-23.427,40.616634,-74.245423,-23.427,-1.642,-7.490,255.379,-19.921,-4.810,0.673,0.085,0.069,0.105,0.02,0.019,0.047
3,576186.01601,17.861,563827.560,4496474.853,-23.424,40.616634,-74.245424,-23.424,-1.636,-7.496,255.379,-19.927,-4.810,0.676,0.085,0.069,0.105,0.02,0.019,0.047
4,576186.02101,17.963,563827.461,4496474.828,-23.421,40.616634,-74.245425,-23.421,-1.632,-7.502,255.378,-19.933,-4.809,0.679,0.085,0.069,0.105,0.02,0.019,0.047
